In [9]:
!apt-get install unrar


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [18]:
import os

# Define the RAR file path and extraction directory
rar_path = "/content/DataSet.rar"
extract_to = "/content/dataset"

# Create the extraction directory if it doesn't exist
os.makedirs(extract_to, exist_ok=True)

# Extract the RAR file
!unrar x -y {rar_path} {extract_to}

# List extracted files to verify
print(f"Files extracted to: {extract_to}")
print(os.listdir(extract_to))


Streaming output truncated to the last 5000 lines.
Extracting  /content/dataset/AugmentMask/16_22.png                         0%  OK 
Extracting  /content/dataset/AugmentMask/16_23.png                         0%  OK 
Extracting  /content/dataset/AugmentMask/16_24.png                         0%  OK 
Extracting  /content/dataset/AugmentMask/16_25.png                         0%  OK 
Extracting  /content/dataset/AugmentMask/16_26.png                         0%  OK 
Extracting  /content/dataset/AugmentMask/16_27.png                         0%  OK 
Extracting  /content/dataset/AugmentMask/16_28.png                         0%  OK 
Extracting  /content/dataset/AugmentMask/16_29.png                         0%  OK 
Extracting  /content/dataset/AugmentMask/16_3.png                          0%  OK 
Extracting  /content/dataset/AugmentMask/16_30.png                         0%  OK 
Extracting  /content/dataset/

In [20]:
import os
import pandas as pd

# Define the paths to the image and mask directories
image_dir = "/content/dataset/AugmentSource"
mask_dir = "/content/dataset/AugmentMask"

# List and sort all image and mask files for consistent mapping
image_files = sorted(os.listdir(image_dir))  # Sorting ensures correct pair matching
mask_files = sorted(os.listdir(mask_dir))   # Sorting ensures correct pair matching

# Verify that the number of images matches the number of masks
if len(image_files) != len(mask_files):
    print(f"Warning: Number of images ({len(image_files)}) and masks ({len(mask_files)}) do not match!")
else:
    print(f"Number of images and masks: {len(image_files)}")

# Ensure consistent file matching by filename (if applicable)
for img, msk in zip(image_files, mask_files):
    if os.path.splitext(img)[0] != os.path.splitext(msk)[0]:
        print(f"Mismatch detected: {img} does not match {msk}")

# Create file paths
image_paths = [os.path.join(image_dir, img) for img in image_files]
mask_paths = [os.path.join(mask_dir, msk) for msk in mask_files]

# Create separate DataFrames for images and masks
image_df = pd.DataFrame({"ImagePath": image_paths})
mask_df = pd.DataFrame({"MaskPath": mask_paths})

# Save the DataFrames to separate CSV files
image_csv_path = "/content/image_paths.csv"
mask_csv_path = "/content/mask_paths.csv"

image_df.to_csv(image_csv_path, index=False, header=False)
mask_df.to_csv(mask_csv_path, index=False, header=False)

print(f"Image CSV file created at: {image_csv_path}")
print(f"Mask CSV file created at: {mask_csv_path}")

# Preview the first few rows of each CSV
print("Image paths preview:")
print(image_df.head())

print("Mask paths preview:")
print(mask_df.head())


Number of images and masks: 2610
Image CSV file created at: /content/image_paths.csv
Mask CSV file created at: /content/mask_paths.csv
Image paths preview:
                                 ImagePath
0   /content/dataset/AugmentSource/0_1.png
1  /content/dataset/AugmentSource/0_10.png
2  /content/dataset/AugmentSource/0_11.png
3  /content/dataset/AugmentSource/0_12.png
4  /content/dataset/AugmentSource/0_13.png
Mask paths preview:
                                MaskPath
0   /content/dataset/AugmentMask/0_1.png
1  /content/dataset/AugmentMask/0_10.png
2  /content/dataset/AugmentMask/0_11.png
3  /content/dataset/AugmentMask/0_12.png
4  /content/dataset/AugmentMask/0_13.png


In [17]:
import shutil

# Path to the folder you want to delete
folder_path = "/content/dataset"

# Delete the folder and all its contents
try:
    shutil.rmtree(folder_path)
    print(f"Deleted folder: {folder_path}")
except FileNotFoundError:
    print(f"Folder not found: {folder_path}")
except Exception as e:
    print(f"Error occurred while deleting the folder: {e}")


Deleted folder: /content/dataset


In [22]:
import pandas as pd
import cv2
import numpy as np
import tensorflow as tf

# Load the CSVs without headers
x_train_paths = pd.read_csv(r'/content/image_paths.csv', header=None)[0]
y_train_paths = pd.read_csv(r'/content/mask_paths.csv', header=None)[0]

# Preprocess data
def preprocess_image(img_path):
    """Preprocess input image: resize, normalize, and add channel dimension."""
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (256, 256))  # Resize to model's input size
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    return img

def preprocess_mask(mask_path):
    """Preprocess mask: resize, normalize, and add channel dimension."""
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (256, 256))
    mask = mask / 255.0
    mask = np.expand_dims(mask, axis=-1)
    return mask

# Batch generator function
def image_mask_generator(image_paths, mask_paths, batch_size=8):
    """Generates batches of images and masks."""
    while True:
        for i in range(0, len(image_paths), batch_size):
            batch_image_paths = image_paths[i:i+batch_size]
            batch_mask_paths = mask_paths[i:i+batch_size]

            # Preprocess batch of images and masks
            batch_images = np.array([preprocess_image(path) for path in batch_image_paths])
            batch_masks = np.array([preprocess_mask(path) for path in batch_mask_paths])

            yield batch_images, batch_masks

# Create the generator for training data
train_generator = image_mask_generator(x_train_paths, y_train_paths, batch_size=8)

# Define the DAG-VNet model
def conv_block(input_tensor, filters, kernel_size=(3, 3), activation='relu'):
    """Convolutional block with two convolution layers and batch normalization."""
    x = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')(input_tensor)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation)(x)
    x = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation)(x)
    return x

def upsample_block(input_tensor, skip_tensor, filters, kernel_size=(3, 3), activation='relu'):
    """Upsampling block with skip connections."""
    x = tf.keras.layers.Conv2DTranspose(filters, kernel_size, strides=(2, 2), padding='same')(input_tensor)
    x = tf.keras.layers.concatenate([x, skip_tensor])
    x = conv_block(x, filters, kernel_size, activation)
    return x

def DAG_VNet(input_shape=(256, 256, 1), num_classes=1):
    """Defines the DAG VNet model."""
    inputs = tf.keras.layers.Input(shape=input_shape)

    # Encoder
    c1 = conv_block(inputs, 64)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

    c2 = conv_block(p1, 128)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

    c3 = conv_block(p2, 256)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

    c4 = conv_block(p3, 512)
    p4 = tf.keras.layers.MaxPooling2D((2, 2))(c4)

    # Bottleneck
    bn = conv_block(p4, 1024)

    # Decoder
    u1 = upsample_block(bn, c4, 512)
    u2 = upsample_block(u1, c3, 256)
    u3 = upsample_block(u2, c2, 128)
    u4 = upsample_block(u3, c1, 64)

    # Output layer
    outputs = tf.keras.layers.Conv2D(num_classes, (1, 1), activation='sigmoid')(u4)

    model = tf.keras.models.Model(inputs, outputs)
    return model

# Instantiate the model
model = DAG_VNet(input_shape=(256, 256, 1), num_classes=1)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Summary of the model
model.summary()

# Train the model using the batch generator
model.fit(train_generator, steps_per_epoch=len(x_train_paths) // 8, epochs=20)

model.save('/content/DAG_VNet_model.keras')


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 256, 256, 1)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_19 (Conv2D)        │ (None, 256, 256, 64)   │            640 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_18    │ (None, 256, 256, 64)   │            256 │ conv2d_19[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_18             │ (None, 256, 256, 64)   │              0 │ batch_normalization_1… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_20 (Conv2D)        │ (None, 256, 256, 64)   │         36,928 │ activation_18[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_19    │ (None, 256, 256, 64)   │            256 │ conv2d_20[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_19             │ (None, 256, 256, 64)   │              0 │ batch_normalization_1… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_4           │ (None, 128, 128, 64)   │              0 │ activation_19[0][0]    │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_21 (Conv2D)        │ (None, 128, 128, 128)  │         73,856 │ max_pooling2d_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_20    │ (None, 128, 128, 128)  │            512 │ conv2d_21[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_20             │ (None, 128, 128, 128)  │              0 │ batch_normalization_2… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_22 (Conv2D)        │ (None, 128, 128, 128)  │        147,584 │ activation_20[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_21    │ (None, 128, 128, 128)  │            512 │ conv2d_22[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_21             │ (None, 128, 128, 128)  │              0 │ batch_normalization_2… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_5      

 Total params: 34,535,745 (131.74 MB)

 Trainable params: 34,523,969 (131.70 MB)

 Non-trainable params: 11,776 (46.00 KB)

Epoch 1/20
326/326 ━━━━━━━━━━━━━━━━━━━━ 178s 484ms/step - accuracy: 0.8640 - loss: 0.3322
Epoch 2/20
326/326 ━━━━━━━━━━━━━━━━━━━━ 188s 484ms/step - accuracy: 0.9019 - loss: 0.2342
Epoch 3/20
326/326 ━━━━━━━━━━━━━━━━━━━━ 157s 482ms/step - accuracy: 0.9091 - loss: 0.2161
Epoch 4/20
326/326 ━━━━━━━━━━━━━━━━━━━━ 157s 482ms/step - accuracy: 0.9214 - loss: 0.1897
Epoch 5/20
326/326 ━━━━━━━━━━━━━━━━━━━━ 158s 483ms/step - accuracy: 0.9311 - loss: 0.1662
Epoch 6/20
326/326 ━━━━━━━━━━━━━━━━━━━━ 157s 483ms/step - accuracy: 0.9444 - loss: 0.1392
Epoch 7/20
326/326 ━━━━━━━━━━━━━━━━━━━━ 157s 483ms/step - accuracy: 0.9543 - loss: 0.1165
Epoch 8/20
326/326 ━━━━━━━━━━━━━━━━━━━━ 158s 483ms/step - accuracy: 0.9603 - loss: 0.1007
Epoch 9/20
326/326 ━━━━━━━━━━━━━━━━━━━━ 158s 483ms/step - accuracy: 0.9650 - loss: 0.0896
Epoch 10/20
326/326 ━━━━━━━━━━━━━━━━━━━━ 158s 483ms/step - accuracy: 0.9711 - loss: 0.0736
Epoch 11/20
326/326 ━━━━━━━━━━━━━━━━━━━━ 158s 484ms/step - accuracy: 0.9764 - loss: 0.0592
Epoch 12

In [23]:
import tensorflow as tf

# Load the saved model
model_path = r'/content/DAG_VNet_model.keras'
model = tf.keras.models.load_model(model_path)
print("Model loaded successfully!")


Model loaded successfully!


In [25]:
import cv2
import numpy as np

def preprocess_image(image_path, target_size=(256, 256)):
    """
    Preprocess the input image: resize, normalize, and add a channel dimension.
    """
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, target_size)
    image = image / 255.0  # Normalize pixel values to [0, 1]
    image = np.expand_dims(image, axis=-1)  # Add channel dimension
    return image

from skimage.measure import regionprops, label

def predict_and_calculate_head_circumference(image_path, model, target_size=(256, 256)):
    """
    Predict the segmentation mask for the given image and calculate head circumference.
    """
    # Preprocess the image
    image = preprocess_image(image_path, target_size)
    image = np.expand_dims(image, axis=0)  # Add batch dimension

    # Predict mask
    pred_mask = model.predict(image)[0, :, :, 0]  # Remove batch dimension
    pred_mask = (pred_mask > 0.5).astype(np.uint8)  # Binary thresholding

    # Label connected regions in the binary mask
    labeled_mask = label(pred_mask)
    regions = regionprops(labeled_mask)

    if regions:
        # Get the perimeter of the largest region (assuming it's the head region)
        largest_region = max(regions, key=lambda r: r.area)
        circumference = largest_region.perimeter
        return circumference
    else:
        print("No region detected!")
        return 0
